In [23]:
import requests
import os
import time
import re
import glob
import matplotlib.pyplot as plt
from scipy import stats
import ast
import json
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import asyncio
import asyncpg

In [24]:
# Получаем текущую дату
current_datetime = datetime.now()
current_datetime = current_datetime - timedelta(days=1)
current_datetime_minus_1_day = (current_datetime).strftime('%Y-%m-%d')

current_datetime_m = current_datetime.replace(day=1).strftime('%Y-%m-%d')

In [25]:
 # Читаем данные с файлов
raw_data = pd.read_csv(f' /sql/state_data/{current_datetime_m}/raw_in_app_events.csv', sep=',')
raw_data_retarget = pd.read_csv(f' /sql/state_data/{current_datetime_m}/raw_in_app_events_retarget.csv', sep=',')
fraud_data = pd.read_csv(f' /sql/state_data/{current_datetime_m}/fraud_post_attribution_installs.csv', sep=',')
offers_by_accounts_offers = pd.read_excel(' /sql/offers_by_accounts.xlsx', sheet_name = 'offers')
offers_by_accounts_VTA = pd.read_excel(' /sql/offers_by_accounts.xlsx', sheet_name = 'VTA')
appmetrica = pd.read_csv(' /sql/state_data/appmetrica/appmetrica.csv')


In [26]:
mode = input(
    '''
    Нажмите q для нового отчёта и отправки на сервер
    или любой символ для заполнения сеток
    '''
)
# stats_offers_mode = input(
#     '''
#     Нажмите w для отправки в Google Sheets
#     или любой символ для заполнения сеток
#     '''
# )

In [27]:
#============================================================================================================================================================= 

# Зарубежные бренды
CIS_brand = ['XM','Binance','Kindred Group','Bet365']

 # Удаляем дубликаты в по этим столбцам (дубликаты уникальных сочетаний этих столбцов)
columns_to_check_offers = ['App_Id', 'Offer_Name', 'Brand', 'Event_Name', 'Media_Source', 'geo', 'cpa', 'curr']
offers_by_accounts_offers = offers_by_accounts_offers.drop_duplicates(subset=columns_to_check_offers)

 # Список апп айди брендов в которых нужно оставить гео только RU
Geo_RU_Only_brand = ['id579961527','ru.stoloto.mobile-allstores','id1038254296','com.carshering','id1265056713','ru.bistrodengi.my']

# ,'Anytime KZ','Anytime BY','id1415669951','kz.anytime.mobile.android','id1231153647','by.drivetime.app'

 # Бренды ЮА в которых нужно оставить гео только RU ('Столото','Delimobil','Bystrodengi')
raw_data = raw_data[~((raw_data['App_ID_clear'].isin(Geo_RU_Only_brand)) & (raw_data['Country_Code']!='RU'))]

 # Бренды ретаргета в которых нужно оставить гео только RU ('Столото ретаргет','Delimobil ретаргет')
raw_data_retarget = raw_data_retarget[~((raw_data_retarget['App_ID_clear'].isin(Geo_RU_Only_brand)) & (raw_data_retarget['Country_Code']!='RU'))]

# Обработка Fonbet. Удаляем строки, в которых по столбцу Sub_Param_1 пусто. Таково условие фонебета по инаппу.
raw_data = raw_data[~((raw_data['Sub_Param_1']=="") & (raw_data['App_ID_clear'].isin(['id1166619854','ru.bkfon-Android'])))]

# Оставляем только нужные гео по юнибету
raw_data = raw_data[~((raw_data['App_ID_clear'].isin(['id905382680','com.unibet.casino','id463335337','com.unibet.unibetpro','id570198108']))\
    & ~(raw_data['Country_Code'].isin(['RO','UK','SE'])))]

#
# print(raw_data[raw_data['App_ID_clear'].isin(['ru.tander.magnit', 'id881463973'])]['App_ID'].count())


In [28]:
 # добавляем в название апп айди обозначение ретаргета так, чтобы затем можно было мерджить данные с офферс бай аккаунта
raw_data_retarget['App_ID'] = raw_data_retarget['App_ID'].apply(lambda x: f"{x[:-2]}reattr{x[-2:]}")

 # Объединяем данные ретаргета и ЮА
raw_data = pd.concat([raw_data, raw_data_retarget], axis=0, ignore_index=True)

# Изменяем формат полей, чтобы затем вычесть их друг от друга и получить окно атрибуции
raw_data['Event_Time'] = pd.to_datetime(raw_data['Event_Time'])
raw_data['Install_Time'] = pd.to_datetime(raw_data['Install_Time'])
raw_data['Atribution'] = (raw_data['Event_Time'] - raw_data['Install_Time']).dt.days

 # Делаем сортировку данных по дате конверсии, чтобы затем удалить фрод и окно атрибуции в нашу пользу
raw_data.sort_values(by='Atribution', ascending=True, inplace=True)

raw_data = raw_data[raw_data['Partner'].isin(['Wakeapp','wakeapppartners','hrngloballcy118', 'adpromarkebg511', 'wakerockmobile', 'mobiadsru988'])] 
fraud_data = fraud_data[fraud_data['Partner'].isin(['Wakeapp','wakeapppartners','hrngloballcy118', 'adpromarkebg511', 'wakerockmobile', 'mobiadsru988'])]

# Обрабатываем поле для дальнейшего удаления дубликатов
raw_data['AppsFlyer_ID'] = raw_data['AppsFlyer_ID'].str.strip()
raw_data['AppsFlyer_ID'] = raw_data['AppsFlyer_ID'].str.replace(r'\s+', '', regex=True)
raw_data['AppsFlyer_ID'] = raw_data['AppsFlyer_ID'].str.strip().str.lower()

# Обрабатываем поле для дальнейшего удаления дубликатов
fraud_data['AppsFlyer_ID'] = fraud_data['AppsFlyer_ID'].str.strip()
fraud_data['AppsFlyer_ID'] = fraud_data['AppsFlyer_ID'].str.replace(r'\s+', '', regex=True)
fraud_data['AppsFlyer_ID'] = fraud_data['AppsFlyer_ID'].str.strip().str.lower()

# Удаляем Дубликаты
raw_data = raw_data.dropna(subset=['AppsFlyer_ID'])
raw_data = raw_data.drop_duplicates(subset=['AppsFlyer_ID','Event_Name','App_ID_clear'])
fraud_data = fraud_data.dropna(subset=['AppsFlyer_ID'])
fraud_data = fraud_data.drop_duplicates(subset=['AppsFlyer_ID','Event_Name','App_ID_clear'])

#raw_data.to_csv(' /sql/state_data/raw_data.csv', index=False)
#fraud_data.to_csv(' /sql/state_data/fraud_data.csv', index=False)

#raw_data.to_csv(' /sql/state_data/raw_data_without_duplicates1.csv', index=False)

# Отмечаем фрод
raw_data['Fraud'] = raw_data['AppsFlyer_ID'].isin(fraud_data['AppsFlyer_ID']).astype(int)
raw_data_with_fraud = raw_data

# Переименовывается поле
offers_by_accounts_VTA.rename(columns={'App_Id':'App_ID'}, inplace=True)

# Происходит объединение наших данных с листом окна атрибуции, чтобы затем убрать данные, которые больше окна атрибуции
raw_data_VTA_merge = pd.merge(raw_data_with_fraud, offers_by_accounts_VTA, on='App_ID', how = 'left')

# Удаляем дубли по полю аф айди
raw_data_VTA_merge['AppsFlyer_ID'] = raw_data_VTA_merge['AppsFlyer_ID'].drop_duplicates()

# Удаляем данные переходящие за окно атрибуции 
#raw_data_VTA_merge.to_csv(' /sql/state_data/raw_data_VTA_merge.csv', index=False)
raw_data_VTA_merge = raw_data_VTA_merge[raw_data_VTA_merge['Atribution'] < raw_data_VTA_merge['VTA']]

# Переименовывается поле
raw_data_VTA_merge.rename(columns={'Country_Code':'geo'}, inplace=True)
# Переименовывается поле
offers_by_accounts_offers.rename(columns={'App_Id':'App_ID'}, inplace=True)

# У российских брендов в гео все заменяется на RU в двух таблицах, чтобы затем таблицы правильно сопоставлялись без потери данных,
#  так как объединение затем будет в том числе по ключу 
offers_by_accounts_offers['geo'] = np.where(offers_by_accounts_offers['Brand'].isin(CIS_brand), offers_by_accounts_offers['geo'],'RU')
raw_data_VTA_merge['geo'] = np.where(raw_data_VTA_merge['Brand'].isin(CIS_brand), raw_data_VTA_merge['geo'],'RU')



In [29]:
#============================================================================================================================================================
# Дополнительная обработка брендов 

# Delimobil
raw_data_VTA_merge = raw_data_VTA_merge[~((raw_data_VTA_merge['Brand'].isin(['Delimobil', 'Delimobil ретаргет'])) & \
    (raw_data_VTA_merge['Is_Primary_Attribution'] == False))]

In [30]:
# Закомменченный код ниже нужен для проверки, что сопоставляет выгрузка по апи с офферс бай аккаунт а что нет - по значению only_left поля _merge, 
# можно понять что не сопоставляется, и не идет дальше в стату
raw_data_offers_merge = pd.merge(raw_data_VTA_merge, offers_by_accounts_offers, on=['App_ID','Event_Name','geo'], how = 'left', indicator=True)
#raw_data_offers_merge.to_csv(' /sql/state_data/raw_data_offers_merge.csv')
#raw_data_VTA_merge.to_csv(' /sql/state_data/raw_data_VTA_merge_1.csv', index=False)

# В значениях поля Event_Time оставляем только год месяц и день, без времени
raw_data_VTA_merge['Event_Time'] = pd.to_datetime(raw_data_VTA_merge['Event_Time']).dt.date

# Оставляем нужные столбцы для дальнейшей группировки
raw_data_VTA_merge = raw_data_VTA_merge[['Partner','Platform','Media_Source','geo','App_ID','Event_Name','Event_Time','Offer_Name','Brand','Fraud']]

# Группируем данные чтобы избежать дублирования данных при мердже
raw_data_VTA_merge = raw_data_VTA_merge.groupby(['Partner','Platform', 'Media_Source','geo','App_ID',\
    'Event_Name','Event_Time','Offer_Name','Brand'], as_index=False).agg(Conversion=('App_ID', 'count'), Fraud=('Fraud', lambda x: x.sum()))

# Объединение по ключам наших данных уже с учетом окна атрибуции с офферс бай аккаунт, чтобы вытащить ставки, бренд,
# менеджера и очистка от данных, которых нет в офферс бай аккаунт 
raw_data_offers_merge = pd.merge(raw_data_VTA_merge, offers_by_accounts_offers, on=['App_ID','Event_Name','geo'], how = 'left')

# raw_data_offers_merge.to_csv(' /sql/state_data/raw_data_offers_merge.csv', index=False)

In [31]:
 
# Оставляем необходимые колонки
raw_data_offers_merge = raw_data_offers_merge[['Partner', 'Media_Source_x',\
    'Platform','App_ID', 'Event_Name', 'Event_Time',\
    'Offer_Name_y', 'Brand_y', 'cpa', 'curr', 'manager','geo','Fraud','Conversion']]

# Переименновываем колонки после мерджа
raw_data_offers_merge.rename(columns={'Media_Source_x':'Media_Source', 'Offer_Name_y':'Offer_Name', 'Brand_y':'Brand'}, inplace=True)

# В значениях поля Event_Time оставляем только год месяц и день, без времени
raw_data_offers_merge['Event_Time'] = pd.to_datetime(raw_data_offers_merge['Event_Time']).dt.date


In [32]:
# Пиды баинга (их нужно удалять только из аккаунта WA)
lst_of_buying_pids = [
    "affiliate", "Apple Search Ads", 
    "bigoads_int", "bigo_int", "blog", "bytedanceglobal_int", "dv360", "dv360_int", "Exceeded_MediaSource_Limit", "Facebook Ads", 
    "facebook2", "facebook_int", "Facebook_int", "fb", "googleadwords_int", "google_int", "mail.ru_int", "None", "Organic",
    "partner", "prelend", "snapchat_int", "taboola_int", "Telegram", "u", "union_fb", "union_google", "Wakeapp", "WakeApp_FB", 
    "yandexdirect_int",  "Blog", "telegram", "meta360s_int", "Blog1", "Blog2", "Blog3", "Blog5", "Blog 4", 
    "Blog6", "huaweiadsglobal_int", "hybrid_int", "dataliftretargeting_int", "liftoff_int", "moloco_int", "appier_int", "appreciateappoffers_int", 
    "wakeappdsp_int", "thetradedesk_int", "yahoogemini_int", "applovin_int", "slikklabs_int",  "adcolony_int", "jampp_int","jampp_int" 
    "personalyrtb_int",  "unity_int", 'app7', 'app8', 'app9', 'Blog7',"tjzymob_int", "cherishads_int", "appnext_int",
    "decho_int" , "unityads_int","Mintegral", "mintegral", "mintegral_int", "vungle_int", 'unityads_int',"ironsource_int", "unityads_int", "xiaomiglobal_int"

]


In [33]:

# Тут устраняются пиды баинга из выгрузок аккаунта Wakeapp (WA), потому что именно тут это является баингом
raw_data_offers_merge = raw_data_offers_merge[~((raw_data_offers_merge['Partner'] == 'Wakeapp') & \
    (raw_data_offers_merge['Media_Source'].isin(lst_of_buying_pids)))]

# Выбираем нужные нам поля
raw_data_offers_merge = raw_data_offers_merge[['Partner', 'Media_Source', 'geo',\
    'Platform','App_ID', 'Event_Name','Event_Time',\
    'Offer_Name', 'Brand', 'cpa', 'curr','manager','Fraud','Conversion']]

# Устраняются пробелы в поле медиа сорс
raw_data_offers_merge['Media_Source'] = raw_data_offers_merge['Media_Source'].str.strip()

# Оставляем только те строки, в которых гео не равно RU
dozap_geo = raw_data_offers_merge[raw_data_offers_merge['geo'] != 'RU'] 
# Оставляем только те строки, в которых гео равно RU
dozap_no_geo = raw_data_offers_merge[raw_data_offers_merge['geo'] == 'RU']

# Выбираем нужные нам поля
dozap_geo_ = dozap_geo[['Offer_Name','Media_Source','Event_Name','geo','Brand']]
# Выбираем нужные нам поля
dozap_no_geo_ = dozap_no_geo[['Offer_Name','Media_Source','Event_Name','geo','Brand']]

# Группируем данные для сокращения объемов данных для дальнеших операций 
merge_geo_dozap = dozap_geo_.groupby(['Offer_Name','Media_Source','Event_Name','geo'], as_index=False).count()
# Группируем данные для сокращения объемов данных для дальнеших операций 
merge_dozap = dozap_no_geo_.groupby(['Offer_Name','Media_Source','Event_Name','geo'], as_index=False).count()


In [34]:

# Читаем данные из листа файла по заполненным сеткам зарубежных брендов
Networks_geo = pd.read_excel(' /sql/networks.xlsx', sheet_name = 'Networks_Geo')
# Читаем данные из листа файла по заполненным сеткам российских брендов
Networks = pd.read_excel(' /sql/networks.xlsx', sheet_name = 'Networks')

# Тут создается список названий полей, которые дальше будут использованы для удаления дублей
columns_to_check_networks = ['Offer_Name', 'Media_Source', 'network_name', 'affiliate', 'Event_Name', 'payout', 'geo']

# Тут удаляются дубли по полям которые переданы переменной (удаляются дублирующие значения именно по сочетанию значений всех строк этих полей)
Networks_geo = Networks_geo.drop_duplicates(subset=columns_to_check_networks)
Networks = Networks.drop_duplicates(subset=columns_to_check_networks)

# Объединение данных из нашей выгрузи и данных в нетворкс, чтобы наши данные получили ставки, название сетки и афилейта.
raw_data_Networks_geo = pd.merge(merge_geo_dozap, Networks_geo, on=['Offer_Name', 'Media_Source', 'geo','Event_Name',], how = 'left', indicator=True)
raw_data_Networks = pd.merge(merge_dozap, Networks, on=['Offer_Name', 'Media_Source','Event_Name',], how = 'left', indicator=True)

# Оставляем только те значения, которые не сопоставились при объединении - те которых нет в нетворксе
raw_data_Networks_geo_dozap = raw_data_Networks_geo[raw_data_Networks_geo['_merge'] == 'left_only']
raw_data_Networks_dozap = raw_data_Networks[raw_data_Networks['_merge'] == 'left_only']

# Объединяем данные с гео и без гео, которые не сопоставились 
for_dozap = pd.concat([raw_data_Networks_geo_dozap, raw_data_Networks_dozap], axis = 0, ignore_index=True)
for_dozap.rename(columns={'Event_Name_x':'Event_Name'}, inplace=True)

# оставляем нужные нам столбцы в несопоставиленных данных.
for_dozap = for_dozap[['Offer_Name', 'Media_Source', 'network_name', 'affiliate', 'Event_Name', 'payout', 'geo']]
for_dozap['geo'] = for_dozap['geo'].replace(['', 'NAN'], 'RU').fillna('RU')

# Выгружаем несопоставленные данные, для заполнения нетворкса. 
for_dozap.to_excel(' /sql/state_data/for_dozap.xlsx', index=False)

# Тут мы выделяем данные которые сопоставились, и отправляем их дальше по обработке 
raw_data_Networks_geo_good = raw_data_Networks_geo[raw_data_Networks_geo['_merge'] == 'both']
raw_data_Networks_good = raw_data_Networks[raw_data_Networks['_merge'] == 'both']

# Объединяем даные с гео и без гео, переименовываем столбцы, оставляем только нужные нам. Дальше мы будем эти данные объединять с нашими сырыми.
dozap_good = pd.concat([raw_data_Networks_geo_good, raw_data_Networks_good], axis = 0, ignore_index=True)
dozap_good.rename(columns={'Event_Name_x':'Event_Name'}, inplace=True)
dozap_good = dozap_good[['Offer_Name', 'Media_Source', 'network_name', 'affiliate', 'Event_Name', 'payout', 'geo']]
# Создаем список столбцов которые пойдут для удаленния дубликатов
columns_to_check = ['Offer_Name', 'Media_Source', 'network_name', 'affiliate', 'Event_Name', 'payout', 'geo']
dozap_good = dozap_good.drop_duplicates(subset=columns_to_check)

# Вместо пустых строк, вставляем гео RU 
dozap_good['geo'] = dozap_good['geo'].replace(['', 'NAN'], 'RU').fillna('RU')

In [35]:
# Здесь группируются наши более сырые данные, не прошедшие через прошлую ячейку - где данные отправились в дозаполнение сеток.
dozap_good_geo_group = dozap_geo.groupby(['Event_Time','Partner','Brand','Offer_Name','App_ID','Platform','Media_Source','Event_Name',\
    'cpa', 'curr', 'manager','geo'], as_index=False).agg(Conversion=('Conversion', 'sum'), Fraud=('Fraud', 'sum'))

dozap_good_group = dozap_no_geo.groupby(['Event_Time','Partner','Brand','Offer_Name','App_ID','Platform','Media_Source','Event_Name',\
    'cpa', 'curr', 'manager','geo'], as_index=False).agg(Conversion=('Conversion', 'sum'), Fraud=('Fraud', 'sum'))


# dozap_good_geo_group.to_csv(' /sql/state_data/dozap_good_geo_group.csv', index=False)

#Объединяем наши сгруппированные данные с теми что со ставками
dozap_good_merge_geo = pd.merge(dozap_good_geo_group, dozap_good, on=['Offer_Name', 'Media_Source', 'Event_Name','geo'], how='left', indicator='merge_status')

dozap_good_merge = pd.merge(dozap_good_group, dozap_good, on=['Offer_Name', 'Media_Source','Event_Name','geo'], how='left', indicator='merge_status')

# dozap_good_merge_geo.to_csv(' /sql/state_data/dozap_good_geo_group.csv', index=False)

# Объединияем наши данные с гео и без гео
for_stats = pd.concat([dozap_good_merge_geo, dozap_good_merge], axis = 0, ignore_index=True)


# Оставляем только те, которые которые конектнулись при мердже
for_stats = for_stats[for_stats['merge_status'] == 'both']

# Оставляем нужные нам строки только
for_stats = for_stats[['Event_Time','Partner','Brand','Offer_Name','App_ID','Platform', 'Media_Source',\
     'Event_Name', 'manager','cpa', 'curr', 'affiliate', 'network_name', 'payout','geo','Conversion','Fraud']]

# Заполняем пустые значения по полю гео, превращая это в RU 
for_stats['geo'] = for_stats['geo'].replace(['', 'NAN'], 'RU').fillna('RU')




In [36]:
#============================================================================================================================================================================
# Фонбет (Fonbet) считается

# Оставляем только фонбет где сорсы adcolony
for_stats = for_stats[(for_stats['Brand'] != 'Фонбет') | ((for_stats['Brand'] == 'Фонбет') & (for_stats['Media_Source'].isin(['adcolony_int', 'adcolony'])))]

# выставляем ставку фонбета где сорсы adcolony
for_stats['cpa'] = np.where(
    ((for_stats['Brand'] == 'Фонбет') & 
     (for_stats['Media_Source'].isin(['adcolony_int', 'adcolony'])) & 
     ((for_stats['Event_Name'] == 'af_baseline2'))), 
    16000, 
    for_stats['cpa']
)

for_stats['cpa'] = np.where(
    ((for_stats['Brand'] == 'Фонбет') & 
     (for_stats['Media_Source'].isin(['adcolony_int', 'adcolony'])) & 
     ((for_stats['Event_Name'] == 'af_ftd'))), 
    10000, 
    for_stats['cpa']
)


In [37]:
# Добавляем данные из аппметрики
for_stats = pd.concat([for_stats, appmetrica], axis=0, ignore_index=True)


In [38]:
# Выгружаем данные из файла с ставками и валютами
currencies = pd.read_excel(r"G:\.shortcut-targets-by-id\1-0y0YvUdxNOM5ZnhM4sE0iJVter1OKIh\тест\ИнАпп\ЦБ инапп.xlsm").tail(1).reset_index(drop=True)

# Тут мы обходим по валютам и ставкам которые не в евро, и соответственно переводим в евро
# Преобразуем 'curr' в массив для работы с масками
def current_eur(row):
    if row['curr'] == 'RUB':
        return row['cpa'] / currencies['EUR'].iloc[0]
    elif row['curr'] == 'USD':
        return row['cpa'] / currencies['USD-EUR'].iloc[0]
    elif row['curr'] == 'GBP':
        return row['cpa'] * currencies['GBP-EUR'].iloc[0]
    else:
        return row['cpa']  # Если валюта уже EUR

# Создаем столбец в котором переводим 
for_stats['cpa_eur'] = for_stats.apply(current_eur, axis=1)

# Применяем множители на основе значений в 'curr'
for_stats['payout_eur'] = for_stats['payout']/currencies['USD-EUR'].iloc[0]


In [39]:
# Создаем чистые ивенты
for_stats['Pure_Conversion'] = for_stats['Conversion'] - for_stats['Fraud']
# Чистые ивенты которые пустые - те что из аппметрики ибо там нет фрода, заполняем данными из Conversion
for_stats['Pure_Conversion'] = for_stats['Pure_Conversion'].fillna(for_stats['Conversion'])
# Создаем евро бюджет, затем расходы, и затем профит
for_stats['Budget'] = for_stats['cpa_eur'] * for_stats['Pure_Conversion']
for_stats['Cost'] = for_stats['payout_eur'] * for_stats['Pure_Conversion']
for_stats['Profit'] = for_stats['Budget'] - for_stats['Cost']

# Оставляем нужные нам поля
for_stats = for_stats[['Event_Time','Partner','Brand','Offer_Name','App_ID','Platform', 'Media_Source',\
    'Event_Name', 'manager','cpa', 'curr', 'cpa_eur', 'affiliate', 'network_name', 'payout','payout_eur',\
    'geo','Conversion','Fraud','Pure_Conversion','Budget','Cost','Profit']]

# Формируем переменную в которую закидываем строки, которые содержат 0 или пустые по полям Cpa_eur - ставка которую платят нам в евро, и Payout_eur то что платим мы в евро
for_stats_without_cpa_payout = for_stats[((for_stats['cpa_eur']== 0) | (for_stats['payout_eur']== 0) |\
    (for_stats['cpa_eur']== 0.0) | (for_stats['payout_eur']== 0.0) | (for_stats['payout_eur']== "") | (for_stats['cpa_eur']== "")\
    |(for_stats['payout_eur']== " ") | (for_stats['cpa_eur']== " "))]

# Сохраняем данные, чтобы изучить и устранить проблему либо в нетворкс либо в офферс бай аккаунт
# for_stats_without_cpa_payout.to_csv(' /sql/state_data/for_stats_without_cpa_payout.csv', index=False)



In [40]:
#===================================================================================================================================================================
# Тут исключаем поля которые по ставкам равны нулю
#for_stats = for_stats[~((for_stats['cpa_eur']== 0) | (for_stats['payout_eur']== 0) & (for_stats['Brand']!= 'Bet365'))]

# Тут обработка Фонбета обнуляем конверсии по регам, за которые нам не платят
for_stats['Pure_Conversion'] = np.where((for_stats['Brand']=='Фонбет') & (for_stats['Event_Name']=='af_reg'), \
    for_stats['Pure_Conversion']== 0, for_stats['Pure_Conversion'])

for_stats['Conversion'] = np.where((for_stats['Brand']=='Фонбет') & (for_stats['Event_Name']=='af_reg'), \
    for_stats['Conversion']== 0, for_stats['Conversion'])

for_stats['Fraud'] = np.where((for_stats['Brand']=='Фонбет') & (for_stats['Event_Name']=='af_reg'), \
    for_stats['Fraud']== 0, for_stats['Fraud'])

for_stats['Pure_Conversion'] = np.where(for_stats['Brand'] == 'Bet365',0, for_stats['Pure_Conversion'])
for_stats['Conversion'] = np.where(for_stats['Brand'] == 'Bet365',0, for_stats['Conversion'])

# Тут происходит фильтрация если в течении месяца какой то бренд стопнулся

# Преобразуем Event_Time в тип datetime64[ns]
for_stats['Event_Time'] = pd.to_datetime(for_stats['Event_Time'])

# Фильтрация
filter_date = pd.to_datetime('2024-12-13')
filter_date_from_20 = pd.to_datetime('2025-01-19')
filter_date_from_5 = pd.to_datetime('2025-02-04')

for_stats = for_stats[~((for_stats['Brand'] == 'Столото') & (for_stats['Event_Time'] < filter_date_from_20))]
for_stats = for_stats[~((for_stats['Brand'] == 'KazanExpress') & (for_stats['Event_Time'] > filter_date))]
for_stats = for_stats[~((for_stats['Brand'] == 'Фонбет') & (for_stats['Event_Time'] <= filter_date_from_5) \
                        & (for_stats['Event_Name'] == 'af_ftd'))]
for_stats = for_stats[~((for_stats['Brand'] == 'Фонбет') & (for_stats['Event_Time'] > filter_date_from_5) \
                        & (for_stats['Event_Name'] == 'af_baseline2'))]


# В значениях поля Event_Time оставляем только год месяц и день, без времени
for_stats['Event_Time'] = pd.to_datetime(for_stats['Event_Time']).dt.date

for_stats['comand_geo'] = np.where(for_stats['geo']== 'RU', 'RU', 'Foreign')

for_stats.to_csv(' /sql/state_data/for_stats.csv',encoding='utf-8', index=False)

In [41]:
# Тут у нас отрпвляются данные на сервер

# Сперва данные оттуда за текущий месяц удаляются=========================================================
import nest_asyncio
nest_asyncio.apply()  # Для работы в Jupyter Notebook

# Асинхронная часть: удаление данных
async def main():
    if mode == 'q':
        with SSHTunnelForwarder(    
            ('),
            ssh_username="",
            ssh_password="",
            remote_bind_address=('', )
        ) as server:
            server.start()
            local_port = server.
            
            # Подключение к базе данных через asyncpg
            conn = await asyncpg.connect(
                database="",
                user="",
                host="",
                password="",
                port=local_port
            )
           
            # Удаление данных
            async with conn.transaction():
                await conn.execute(
                    '''
                    DELETE FROM "Stats_new"
                    WHERE DATE_TRUNC('month', "Event_Time") = DATE_TRUNC('month', now() - interval '1' day)
                    '''
                )
                print("Данные успешно удалены.")

            # Закрытие подключения
            await conn.close()
        server.stop()

# Запуск программы
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

# Затем, второй транзакцией мы данные добавляем =============================================================
if mode == 'q':
    # Подключение к серверу и базе данных
    with SSHTunnelForwarder(
        ('', ),  # Remote server IP and SSH port
        ssh_username="c",
        ssh_password="",
        remote_bind_address=('', )
    ) as server:

        server.start()
        
        # Получаем локальный порт
        local_port = str(server.)
        engine = create_engine(f'postgresql:)

        # Загрузка новых данных в таблицу
        for_stats.to_sql('Stats_new', con=engine, if_exists='append', index=False)

        print("Данные успешно обновлены.")

    server.stop()

Данные успешно удалены.
Данные успешно обновлены.


In [42]:
# Считается месячная стата
for_stats_month = for_stats.groupby(
    ['Brand']
).agg(
    Conversion=('Pure_Conversion', 'sum'),
    Fraud=('Fraud', 'sum'),
    Budget=('Budget', 'sum'),
    Cost=('Cost', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()
# Считаем процент фрода в поле фрод
for_stats_month['Fraud'] = (for_stats_month['Fraud'] / for_stats_month['Conversion'])
# Считаем Рои
for_stats_month['ROI'] = (for_stats_month['Profit'] / for_stats_month['Cost'])
# Обрабатываем Рои для бет 365
for_stats_month['ROI'].replace([float('inf'), -float('inf')], 0, inplace=True)

# Считаем итоги
totals = {
    'Brand': 'Total',
    'Conversion': for_stats_month['Conversion'].sum(),
    'Fraud': (for_stats_month['Fraud'] * for_stats_month['Conversion']).sum() / for_stats_month['Conversion'].sum(),
    'Budget': for_stats_month['Budget'].sum(),
    'Cost': for_stats_month['Cost'].sum(),
    'Profit': for_stats_month['Profit'].sum(),
    'ROI': for_stats_month['Profit'].sum() / for_stats_month['Cost'].sum()
}
# Добавляем строку с итогами
for_stats_month = pd.concat([for_stats_month, pd.DataFrame([totals])], ignore_index=True)





for_stats_ = for_stats[for_stats['Event_Time'] == pd.Timestamp(current_datetime_minus_1_day).date()]
print()
# Считается стата на прошлый день
for_stats_day = for_stats_.groupby(
    ['Brand']
).agg(
    Conversion=('Pure_Conversion', 'sum'),    
    Fraud=('Fraud', 'sum'),
    Budget=('Budget', 'sum'),
    Cost=('Cost', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()
# Считаем процент фрода в поле фрод
for_stats_day['Fraud'] = (for_stats_day['Fraud'] / for_stats_day['Conversion'])
# Считаем Рои
for_stats_day['ROI'] = (for_stats_day['Profit'] / for_stats_day['Cost'])
# Обрабатываем Рои для бет 365
for_stats_day['ROI'].replace([float('inf'), -float('inf')], 0, inplace=True)

totals = {
    'Brand': 'Total',
    'Conversion': for_stats_day['Conversion'].sum(),
    'Fraud': (for_stats_day['Fraud'] * for_stats_day['Conversion']).sum() / for_stats_day['Conversion'].sum(),
    'Budget': for_stats_day['Budget'].sum(),
    'Cost': for_stats_day['Cost'].sum(),
    'Profit': for_stats_day['Profit'].sum(),
    'ROI': for_stats_day['Profit'].sum() / for_stats_day['Cost'].sum()
}
# Добавляем строку с итогами
for_stats_day = pd.concat([for_stats_day, pd.DataFrame([totals])], ignore_index=True)



# Считается дневная
for_stats_d = for_stats.groupby(
    ['Event_Time','Brand']
).agg(
    Conversion=('Pure_Conversion', 'sum'),
    Fraud=('Fraud', 'sum'),
    Budget=('Budget', 'sum'),
    Cost=('Cost', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()
# Считаем процент фрода в поле фрод
for_stats_d['Fraud'] = (for_stats_d['Fraud'] / for_stats_d['Conversion'])
# Считаем Рои
for_stats_d['ROI'] = (for_stats_d['Profit'] / for_stats_d['Cost'])
# Обрабатываем Рои для бет 365
for_stats_d['ROI'].replace([float('inf'), -float('inf')], 0, inplace=True)

# Выгружаем стату
with pd.ExcelWriter(' /sql/state_data/stats.xlsx') as path:
    for_stats_month.to_excel(path, sheet_name ='state month', index=False)
    for_stats_day.to_excel(path, sheet_name =f'state day {current_datetime_minus_1_day}', index=False)
    for_stats_d.to_excel(path, sheet_name ='state everyday', index=False)

In [43]:
# Считается месячная стата CIS

for_stats_CIS = for_stats[for_stats['geo']=='RU']
for_stats_month = for_stats_CIS.groupby(
    ['Brand']
).agg(
    Conversion=('Pure_Conversion', 'sum'),
    Fraud=('Fraud', 'sum'),
    Budget=('Budget', 'sum'),
    Cost=('Cost', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()
# Считаем процент фрода в поле фрод
for_stats_month['Fraud'] = (for_stats_month['Fraud'] / for_stats_month['Conversion'])
# Считаем Рои
for_stats_month['ROI'] = (for_stats_month['Profit'] / for_stats_month['Cost'])

# Считаем итоги
totals = {
    'Brand': 'Total',
    'Conversion': for_stats_month['Conversion'].sum(),
    'Fraud': (for_stats_month['Fraud'] * for_stats_month['Conversion']).sum() / for_stats_month['Conversion'].sum(),
    'Budget': for_stats_month['Budget'].sum(),
    'Cost': for_stats_month['Cost'].sum(),
    'Profit': for_stats_month['Profit'].sum(),
    'ROI': for_stats_month['Profit'].sum() / for_stats_month['Cost'].sum()
}
# Добавляем строку с итогами
for_stats_month = pd.concat([for_stats_month, pd.DataFrame([totals])], ignore_index=True)





for_stats_ = for_stats_CIS[for_stats_CIS['Event_Time'] == pd.Timestamp(current_datetime_minus_1_day).date()]

# Считается стата на прошлый день
for_stats_day = for_stats_.groupby(
    ['Brand']
).agg(
    Conversion=('Pure_Conversion', 'sum'),    
    Fraud=('Fraud', 'sum'),
    Budget=('Budget', 'sum'),
    Cost=('Cost', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()
# Считаем процент фрода в поле фрод
for_stats_day['Fraud'] = (for_stats_day['Fraud'] / for_stats_day['Conversion'])
# Считаем Рои
for_stats_day['ROI'] = (for_stats_day['Profit'] / for_stats_day['Cost'])

totals = {
    'Brand': 'Total',
    'Conversion': for_stats_day['Conversion'].sum(),
    'Fraud': (for_stats_day['Fraud'] * for_stats_day['Conversion']).sum() / for_stats_day['Conversion'].sum(),
    'Budget': for_stats_day['Budget'].sum(),
    'Cost': for_stats_day['Cost'].sum(),
    'Profit': for_stats_day['Profit'].sum(),
    'ROI': for_stats_day['Profit'].sum() / for_stats_day['Cost'].sum()
}
# Добавляем строку с итогами
for_stats_day = pd.concat([for_stats_day, pd.DataFrame([totals])], ignore_index=True)



# Считается дневная
for_stats_d = for_stats_CIS.groupby(
    ['Event_Time','Brand']
).agg(
    Conversion=('Pure_Conversion', 'sum'),
    Fraud=('Fraud', 'sum'),
    Budget=('Budget', 'sum'),
    Cost=('Cost', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()
# Считаем процент фрода в поле фрод
for_stats_d['Fraud'] = (for_stats_d['Fraud'] / for_stats_d['Conversion'])
# Считаем Рои
for_stats_d['ROI'] = (for_stats_d['Profit'] / for_stats_d['Cost'])


# Выгружаем стату
with pd.ExcelWriter(' /sql/state_data/stats_CIS.xlsx') as path:
    for_stats_month.to_excel(path, sheet_name ='state month', index=False)
    for_stats_day.to_excel(path, sheet_name =f'state day {current_datetime_minus_1_day}', index=False)
    for_stats_d.to_excel(path, sheet_name ='state everyday', index=False)

In [44]:
# Считается месячная стата WW

for_stats_WW = for_stats[for_stats['geo']!='RU']
for_stats_month = for_stats_WW.groupby(
    ['Brand']
).agg(
    Conversion=('Pure_Conversion', 'sum'),
    Fraud=('Fraud', 'sum'),
    Budget=('Budget', 'sum'),
    Cost=('Cost', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()
# Считаем процент фрода в поле фрод
for_stats_month['Fraud'] = (for_stats_month['Fraud'] / for_stats_month['Conversion'])
# Считаем Рои
for_stats_month['ROI'] = (for_stats_month['Profit'] / for_stats_month['Cost'])
# Обрабатываем Рои для бет 365
for_stats_month['ROI'].replace([float('inf'), -float('inf')], 0, inplace=True)

# Считаем итоги
totals = {
    'Brand': 'Total',
    'Conversion': for_stats_month['Conversion'].sum(),
    'Fraud': (for_stats_month['Fraud'] * for_stats_month['Conversion']).sum() / for_stats_month['Conversion'].sum(),
    'Budget': for_stats_month['Budget'].sum(),
    'Cost': for_stats_month['Cost'].sum(),
    'Profit': for_stats_month['Profit'].sum(),
    'ROI': for_stats_month['Profit'].sum() / for_stats_month['Cost'].sum()
}
# Добавляем строку с итогами
for_stats_month = pd.concat([for_stats_month, pd.DataFrame([totals])], ignore_index=True)





for_stats_ = for_stats_WW[for_stats_WW['Event_Time'] == pd.Timestamp(current_datetime_minus_1_day).date()]

# Считается стата на прошлый день
for_stats_day = for_stats_.groupby(
    ['Brand']
).agg(
    Conversion=('Pure_Conversion', 'sum'),    
    Fraud=('Fraud', 'sum'),
    Budget=('Budget', 'sum'),
    Cost=('Cost', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()
# Считаем процент фрода в поле фрод
for_stats_day['Fraud'] = (for_stats_day['Fraud'] / for_stats_day['Conversion'])
# Считаем Рои
for_stats_day['ROI'] = (for_stats_day['Profit'] / for_stats_day['Cost'])
# Обрабатываем Рои для бет 365
for_stats_day['ROI'].replace([float('inf'), -float('inf')], 0, inplace=True)

totals = {
    'Brand': 'Total',
    'Conversion': for_stats_day['Conversion'].sum(),
    'Fraud': (for_stats_day['Fraud'] * for_stats_day['Conversion']).sum() / for_stats_day['Conversion'].sum(),
    'Budget': for_stats_day['Budget'].sum(),
    'Cost': for_stats_day['Cost'].sum(),
    'Profit': for_stats_day['Profit'].sum(),
    'ROI': for_stats_day['Profit'].sum() / for_stats_day['Cost'].sum()
}
# Добавляем строку с итогами
for_stats_day = pd.concat([for_stats_day, pd.DataFrame([totals])], ignore_index=True)



# Считается дневная
for_stats_d = for_stats_WW.groupby(
    ['Event_Time','Brand']
).agg(
    Conversion=('Pure_Conversion', 'sum'),
    Fraud=('Fraud', 'sum'),
    Budget=('Budget', 'sum'),
    Cost=('Cost', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()
# Считаем процент фрода в поле фрод
for_stats_d['Fraud'] = (for_stats_d['Fraud'] / for_stats_d['Conversion'])
# Считаем Рои
for_stats_d['ROI'] = (for_stats_d['Profit'] / for_stats_d['Cost'])
# Обрабатываем Рои для бет 365
for_stats_d['ROI'].replace([float('inf'), -float('inf')], 0, inplace=True)

# Выгружаем стату
with pd.ExcelWriter(' /sql/state_data/stats_WW.xlsx') as path:
    for_stats_month.to_excel(path, sheet_name ='state month', index=False)
    for_stats_day.to_excel(path, sheet_name =f'state day {current_datetime_minus_1_day}', index=False)
    for_stats_d.to_excel(path, sheet_name ='state everyday', index=False)